In [22]:
def savingMySQL(NumList, TextList, AuList): #mySQL에 새 소식 저장하기
    import pymysql
    from openpyxl import Workbook
    from openpyxl import load_workbook
    conn = pymysql.connect(host='localhost',user='root',password='123456',db='cnunotifier')
    cursor=conn.cursor()

    cursor.execute("truncate cnu_com_notifier")
    try:
        for i in range(len(NumList)):
            with conn.cursor() as cursor:  #with as 문을 사용하면 close 하지 않아도 알아서 파일을close 해줌
                sql = "insert into cnu_com_notifier values ("+NumList[i]+", '"+TextList[i]+ "', '"+AuList[i]+"')"
                cursor.execute(sql)
                conn.commit()
    finally:
        conn.close()
    return

In [28]:
def getNewsNumberList(): #소식 번호만 리스트로 받아오기
    import pymysql
    from openpyxl import Workbook
    from openpyxl import load_workbook
    conn = pymysql.connect(host='localhost',user='root',password='123456',db='cnunotifier')
    cursor=conn.cursor()
    try:
        with conn.cursor() as cursor:  #with as 문을 사용하면 close 하지 않아도 알아서 파일을close 해줌
            sql = "select number from cnu_com_notifier order by number desc"
            cursor.execute(sql)
            result_list=[]
            for i in range(5):
                result = cursor.fetchone()
                result_list+=list(result)



    finally:
        conn.close()
    return result_list

In [29]:
def printNewNews(num): #num의 개수만큼 새 소식 받아오기

    import pymysql
    from openpyxl import Workbook
    from openpyxl import load_workbook
    conn = pymysql.connect(host='localhost',user='root',password='123456',db='cnunotifier')
    cursor=conn.cursor()
    try:
        with conn.cursor() as cursor:  #with as 문을 사용하면 close 하지 않아도 알아서 파일을close 해줌
            sql = "select * from cnu_com_notifier order by number desc"
            cursor.execute(sql)
            result_list=[]
            for i in range(num):
                result = cursor.fetchone()
                result_list+=list(result)
            print(result_list)
    finally:
        conn.close()
    return

In [25]:
def crawlingCNUcom():
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    html=urlopen("http://computer.cnu.ac.kr/index.php?mid=notice&page=1")
    bsObj=BeautifulSoup(html.read(),"html.parser")
    Num=bsObj.html.body.tbody.findAll("tr")
    NumList=[]
    TextList=[]
    AuList=[]
    for line in Num:
        while len(NumList)<5:
            if '공지' in line.find("td",{"class":"no"}).get_text():
                break
            else:
                NumList+=[line.find("td",{"class":"no"}).get_text().strip()]
                TextList+=[line.find("td", {"class":"title"}).get_text().strip()]
                AuList+=[line.find("td", {"class":"author"}).get_text().strip()]
                
                break
    savingMySQL(NumList,TextList,AuList)
  
   
    return

In [26]:
def findNumOfNew(NewList,OldList):
    if NewList[0]==OldList[0]:#List1이 새거
        print('새로운 소식이 없습니다')
        return 0
    else:
        News=str((int)(NewList[0])-(int)(OldList[0]))
        print('새로운 소식이 '+News+'개 있습니다.')
        return int(News)

In [30]:
import time
while True:
    crawlingCNUcom()
    OldList=getNewsNumberList()
    
    time.sleep(60)
    
    crawlingCNUcom()
    NewList=getNewsNumberList()

    
    numOfNewNews=findNumOfNew(NewList,OldList)
    if numOfNewNews is not 0:
        printNewNews(numOfNewNews)
    
    

KeyboardInterrupt: 